<a href="https://colab.research.google.com/github/emcee21/colabML/blob/master/Sinus_Infection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy.random import randint, uniform
 
import matplotlib.pyplot as plt
 


#@title Example form fields
#@markdown Forms support many types of fields.

# number N of points where a curve is sampled  
SAMPLE_LEN =    64#@param {type: "integer"} 

# number of curves in the training set
SAMPLE_SIZE = 32768   #@param {type: "integer"} 

# least ordinate where to sample
X_MIN = -5.0  #@param {type: "number"} 
# last ordinate where to sample
X_MAX = 5.0   #@param {type: "number"} 


BATCH =        8172#@param {type: "integer"}
EPOCHS =    65536#@param {type: "integer"}

DISCRIMINATOR_DROPOUT_RATE = 0.25  #@param {type: "number"}
GENERATOR_LEAK_RATE = 0.1  #@param {type: "number"}
 
DISCRIMINATOR_LEARNING_RATE = 0.2  #@param {type: "number"}
GENERATOR_LEARNING_RATE = 0.004  #@param {type: "number"}

EPSILON = 0.0000001  #@param {type: "number"}
#@markdown ---




 
# The set of coordinates over which curves are sampled
X_COORDS = np.linspace(X_MIN , X_MAX, SAMPLE_LEN)
 
# The training set
SAMPLE = np.zeros((SAMPLE_SIZE, SAMPLE_LEN))
for i in range(0, SAMPLE_SIZE):
    b = uniform(0.5, 2.0)
    c = uniform(np.math.pi)
    SAMPLE[i] = np.array([np.sin(b*x + c) for x in X_COORDS])
 
# We plot the first 8 curves
fig, axis = plt.subplots(1, 1)
for i in range(8):
    axis.plot(X_COORDS, SAMPLE[i])


plt.show()


In [ ]:

print(f'SAMPLE_LEN={SAMPLE_LEN}')
print(f'SAMPLE_SIZE={SAMPLE_SIZE}')
print(f'X_MIN={X_MIN}')
print(f'X_MAX={X_MAX}')
print(f'BATCH={BATCH}')
print(f'EPOCHS={EPOCHS}')
print(f'DISCRIMINATOR_DROPOUT_RATE={DISCRIMINATOR_DROPOUT_RATE}')
print(f'GENERATOR_LEAK_RATE={GENERATOR_LEAK_RATE}')
print(f'DISCRIMINATOR_LEARNING_RATE={DISCRIMINATOR_LEARNING_RATE}')
print(f'GENERATOR_LEARNING_RATE={GENERATOR_LEARNING_RATE}')
print(f'EPSILON={EPSILON}')



from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU
from keras.optimizers import Adam
 
DROPOUT = Dropout(DISCRIMINATOR_DROPOUT_RATE)        # Empirical hyperparameter
discriminator = Sequential()
discriminator.add(Dense(SAMPLE_LEN))
discriminator.add(DROPOUT)
discriminator.add(Dense(512))
discriminator.add(DROPOUT)
discriminator.add(Dense(1, activation = "sigmoid"))
discriminator.compile(optimizer = Adam(learning_rate=DISCRIMINATOR_LEARNING_RATE, epsilon = EPSILON), loss = "binary_crossentropy", metrics = ["accuracy"])


LEAKY_RELU = LeakyReLU(GENERATOR_LEAK_RATE)   # Empirical hyperparameter
generator = Sequential()
generator.add(Dense(SAMPLE_LEN))
generator.add(LEAKY_RELU)
generator.add(Dense(512))
generator.add(LEAKY_RELU)
generator.add(Dense(SAMPLE_LEN, activation = "tanh"))
#generator.compile(optimizer = Adam(learning_rate=GENERATOR_LEARNING_RATE, epsilon = EPSILON), loss = "mse", metrics = ["accuracy"])
#generator.compile()

gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(optimizer = Adam(learning_rate=GENERATOR_LEARNING_RATE, epsilon = EPSILON), loss = "binary_crossentropy", metrics = ["accuracy"])

NOISE = uniform(X_MIN, X_MAX, size = (SAMPLE_SIZE, SAMPLE_LEN))
ONES = np.ones((SAMPLE_SIZE))
ZEROS = np.zeros((SAMPLE_SIZE))
print("epoch | dis. loss | dis. acc | gen. loss | gen. acc")
print("------+-----------+----------+-----------+----------")
 




BATCHES_PER_EPOCH = (SAMPLE_SIZE // BATCH)
g_losses = np.zeros(([EPOCHS * BATCHES_PER_EPOCH]))
g_accuracy = np.zeros(([EPOCHS * BATCHES_PER_EPOCH]))
d_losses = np.zeros(([EPOCHS * BATCHES_PER_EPOCH]))
d_accuracy = np.zeros(([EPOCHS * BATCHES_PER_EPOCH]))



ax_index = 1
stats_index = 0
for e in range(EPOCHS):
    for k in range(BATCHES_PER_EPOCH):
        # Addestra il discriminatore a riconoscere le sinusoidi vere da quelle prodotte dal generatore
        n = randint(0, SAMPLE_SIZE, size = BATCH)
        # Ora prepara un batch di training record per il discriminatore
        p = generator.predict(NOISE[n])
        #p1 = discriminator.predict(SAMPLE[n])
        x = np.concatenate((SAMPLE[n], p))
        y = np.concatenate((ONES[n], ZEROS[n]))
        d_result = discriminator.train_on_batch(x, y)
        d_losses.put([stats_index],[d_result[0]])
        d_accuracy.put(stats_index,d_result[1])

        #p2 = discriminator.predict(x)
        discriminator.trainable = False
        g_result = gan.train_on_batch(NOISE[n], ONES[n])
        g_losses.put(stats_index, g_result[0])
        g_accuracy.put(stats_index, g_result[1])

        discriminator.trainable = True
        stats_index += 1

    print(f" {e:04n} |  {d_result[0]:.5f}  |  {d_result[1]:.5f} |  {g_result[0]:.5f}  |  {g_result[1]:.5f}")
    # At 3, 13, 23, ... plots the last generator prediction
    if e % 10 == 3:
        #print(f"{generator.metrics_names}")
        #print(f"{discriminator.metrics_names}")
        #print(f"{gan.metrics_names}")
        #print(f"p1 sum={p1.sum()}")
        #print(f"p2 sum={p2.sum()}")

        #gan.summary()

        fig, axs = plt.subplots(3, 1, figsize= (8, 12))

        ax = axs[0]
        ax.plot(X_COORDS, p[-1])
        #print(p[-1])
        ax.xaxis.set_visible(False)
        ax.set_ylabel(f"Epoch: {e}")

#        ax = losses_fig.add_subplot(EPOCHS + 1 / 10 , 1, ax_index)
        ax = axs[1]
        ax.plot(log(d_losses[0:stats_index - 1]))
        ax.plot(log(g_losses[0:stats_index - 1]))
        ax.xaxis.set_visible(False)
        ax.set_ylabel(f"Losses Epoch: {e}")

#        ax = accuracy_fig.add_subplot(EPOCHS + 1 / 10 , 1, ax_index)
        ax = axs[2]
        ax.plot(d_accuracy[0:stats_index - 1])
        ax.plot(g_accuracy[0:stats_index - 1])
        ax.xaxis.set_visible(False)
        ax.set_ylabel(f"Accuracy Epoch: {e}")

        plt.show()
        
        ax_index += 1


# Plots a curve generated by the GAN
y = generator.predict(uniform(X_MIN, X_MAX, size = (1, SAMPLE_LEN)))[0]
ax = fig.add_subplot(EPOCHS + 1 / 10, 1, ax_index)
plt.plot(X_COORDS, y)

